In [1]:
import dask.dataframe as dframe
import pandas as pd
import numpy as np
from sklearn.externals import joblib

pd.set_option('max_rows', 184)
pd.set_option('max_columns', 184)

In [2]:
dys_types = ['norm', 'ds', 'pfl', 'pvt', 'ie']
model_path = 'c:\\Research_data\\VD_models\\'
data_path = 'c:\\Research_data\\Analysis\\pandas_breath_dump_5_1_16.h5'

In [3]:
df = pd.read_hdf(data_path, key='breath_collection')
df = df[(df['breath_character.peak_vol'] < 2000) & (df['breath_character.peak_vol'] > 100) & (df['breath_character.elapse_time'] < 6000)]
df.set_index(['date_time'], inplace=True, append=True, drop=False)

In [4]:
df.shape

(1904166, 184)

In [14]:
unique_id = df.patient_id.unique().tolist()
unique_id

[100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132]

In [6]:
max_min = df.columns[df.columns.str.contains('max_min_analysis')]

In [6]:
exists_df = df.isnull()
exists_df = exists_df[exists_df == True].count()
exists_df[exists_df > 10000].sort_values()

breath_settings.patient_ID           34446
breath_settings.vent_mode            34446
breath_settings.peep                 34446
breath_settings.ramp                 34764
breath_settings.high_paw_alarm       34764
breath_settings.tigger               34764
breath_settings.fio2                 34764
breath_settings.exp_flow             34784
breath_settings.leak                 34785
breath_settings.insp_flow            34793
breath_settings.mean_paw             34794
breath_settings.peak_paw             34797
breath_settings.rr                   34797
breath_settings.vte                  34797
breath_settings.vti                  34797
breath_settings.exp_minute_vol       34798
breath_settings.insp_paw             34846
breath_settings.t_insp               45894
breath_settings.t_exp                45895
breath_settings.min_paw              80863
breath_settings.compliance          144191
breath_settings.i:e                 227346
breath_settings.plat_paw            228679
max_min_ana

In [32]:
for ids in unique_id:
    print(ids)
    
    for types in dys_types: 
        cols = joblib.load(model_path+types+'\\'+types+'_cols.pkl')
        scaler = joblib.load(model_path+types+'\\'+types+'_scaler.pkl')
        clf = joblib.load(model_path+types+'\\'+types+'.pkl')
        cols.append('date_time')

        print(types)
        df = pd.read_hdf('c:\\Research_data\\Analysis\\'+str(ids)+'_pandas_breath_dump_5_1_16.h5', key='breath_collection')
        df = df[(df['breath_character.peak_vol'] < 2000) & (df['breath_character.peak_vol'] > 100) & (df['breath_character.elapse_time'] < 6000)]
        df.set_index('date_time', inplace=True, drop=False)
        
        print(df[cols].isnull().sum().max())
        data = df[cols].dropna(how='any', axis=0)
        print(data.shape)

        labels = data['date_time'].values
        cols.remove('date_time')
        x = data[cols].values

        x = scaler.transform(x)
        y = clf.predict(x)

        analysis_df = pd.DataFrame({types+'_predict':y, 'date_time':labels[:]})
        analysis_df.set_index('date_time', inplace=True, drop=True)

        try:
            df = df.join(analysis_df, how='outer')
        except ValueError as e:
            print('overlap', e)
        df.to_hdf('c:\\Research_data\\Analysis\\'+str(ids)+'_pandas_breath_dump_5_1_16.h5', key='breath_collection', type='Table', append=False)

    

100
norm
1
(209, 86)
overlap columns overlap but no suffix specified: Index(['norm_predict'], dtype='object')
ds
3
(207, 7)
overlap columns overlap but no suffix specified: Index(['ds_predict'], dtype='object')


C:\Users\sottilep\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:1096: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['_id', 'breath_settings.i:e', 'breath_settings.vent_mode']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


pfl
1
(209, 70)
overlap columns overlap but no suffix specified: Index(['pfl_predict'], dtype='object')
pvt
1
(209, 13)
overlap columns overlap but no suffix specified: Index(['pvt_predict'], dtype='object')
ie
1
(209, 34)
overlap columns overlap but no suffix specified: Index(['ie_predict'], dtype='object')
101
norm
42
(43831, 86)
ds
7
(43867, 7)
pfl
43
(43831, 70)
pvt
43
(43831, 13)
ie
43
(43831, 34)
102
norm
0
(1696, 86)
ds
2
(1694, 7)
pfl
0
(1696, 70)
pvt
0
(1696, 13)
ie
0
(1696, 34)
103
norm
3
(36277, 86)
ds
7
(36276, 7)
pfl
6
(36277, 70)
pvt
6
(36277, 13)
ie
6
(36277, 34)
104
norm
1
(346, 86)
ds
1
(346, 7)
pfl
1
(346, 70)
pvt
1
(346, 13)
ie
1
(346, 34)
105
norm
6
(41355, 86)
ds
9
(41357, 7)
pfl
11
(41355, 70)
pvt
11
(41355, 13)
ie
11
(41355, 34)
106
norm
16
(49775, 86)
ds
8
(49785, 7)
pfl
18
(49775, 70)
pvt
18
(49775, 13)
ie
18
(49775, 34)
107
norm
9
(8094, 86)
ds
3
(8101, 7)
pfl
10
(8094, 70)
pvt
10
(8094, 13)
ie
10
(8094, 34)
108
norm
0
(4603, 86)
ds
1
(4602, 7)
pfl
0
(4603, 70

In [33]:
tot_df = pd.DataFrame()
for ids in unique_id:
        print(ids)
        df = pd.read_hdf('c:\\Research_data\\Analysis\\'+str(ids)+'_pandas_breath_dump_5_1_16.h5', key = 'breath_collection',
                  format = 'table')
        tot_df = pd.concat([tot_df, df])
print(tot_df.shape)
tot_df.to_hdf('c:\\Research_data\\Analysis\\pandas_breath_dump_5_1_16.h5', key = 'breath_collection',
                 format = 'table', append = False)

100
101
102
103
104
105
106
107
108
109
110
112
113
114
115
116
117
118
119
120
121
122
124
125
126
127
128
129
130
131
132
(1904166, 189)


In [41]:
tot_df[['ds_predict', 'pfl_predict', 'pvt_predict', 'ie_predict', 'norm_predict']].sum()

ds_predict        59655.0
pfl_predict      399197.0
pvt_predict      591987.0
ie_predict       538548.0
norm_predict    1200052.0
dtype: float64

In [40]:
list(tot_df.columns)

['_id',
 'breath_character.elapse_time',
 'breath_character.end_insp_flow',
 'breath_character.end_insp_paw',
 'breath_character.end_insp_time',
 'breath_character.end_insp_vol',
 'breath_character.end_time',
 'breath_character.exp_time',
 'breath_character.insp_time',
 'breath_character.mean_exp_flow',
 'breath_character.mean_exp_paw',
 'breath_character.mean_insp_flow',
 'breath_character.mean_insp_paw',
 'breath_character.min_exp_flow',
 'breath_character.min_exp_paw',
 'breath_character.min_exp_vol',
 'breath_character.min_flow',
 'breath_character.min_paw',
 'breath_character.min_vol',
 'breath_character.n_dF/dP_exp_max',
 'breath_character.n_dF/dP_insp_max',
 'breath_character.n_dF/dV_exp_max',
 'breath_character.n_dF/dV_insp_max',
 'breath_character.n_dP/dV_exp_max',
 'breath_character.n_dP/dV_insp_max',
 'breath_character.peak_flow',
 'breath_character.peak_paw',
 'breath_character.peak_vol',
 'breath_character.start_time',
 'breath_num',
 'breath_settings.auto_peep',
 'breath_